# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Constants

MODEL_GPT = 'openai/gpt-4o-mini'
MODEL_CLAUDE = 'anthropic/claude-sonnet-4.5'
MODEL_GEMINI = 'google/gemini-2.5-flash-lite'

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
openrouter_url = "https://openrouter.ai/api/v1"

In [ ]:
# Initialization

load_dotenv(override=True)

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
if openrouter_api_key:
    print(f"OpenAI API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenAI API Key not set")


anthropic = OpenAI(api_key=openrouter_url, base_url=anthropic_url)
gemini = OpenAI(api_key=openrouter_url, base_url=gemini_url)
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)

OpenAI API Key exists and begins sk-


In [ ]:
system_message = """
You are a software engineer with 20 years of experience in the industry.
You are going to answer questions about the software development industry.
You will give concise answers which are not too long.
"""

In [ ]:
# Map display names to OpenRouter model IDs
MODEL_MAP = {"GPT": MODEL_GPT, "Claude": MODEL_CLAUDE, "Gemini": MODEL_GEMINI}

model_selector = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")

In [ ]:
def chat(message, history, model):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    model_id = MODEL_MAP.get(model, MODEL_GPT)
    stream = openrouter.chat.completions.create(model=model_id, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
view = gr.ChatInterface(
    fn=chat,
    type="messages",
    additional_inputs=[model_selector],
    title="Technical Q&A Assistant",
    description="Ask technical questions about programming. Choose your model above.",
).launch()